In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
subs=pd.read_pickle('../Datasets/submissions.pickle')

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

finbert_tone = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer,return_all_scores=True)

def finbert_tone_model(txt):
    try: 
        return finbert_tone(txt)
    except:
        return 'no_result'

TAPAS models are not usable since `torch_scatter` can't be loaded. It seems you have `torch_scatter` installed with the wrong CUDA version. Please try to reinstall it following the instructions here: https://github.com/rusty1s/pytorch_scatter.


In [2]:
btc=subs[subs.topic=='Bitcoin']

In [6]:
btc['sent_fbt']=btc.clean_text.progress_apply(finbert_tone_model)

100%|██████████| 527651/527651 [4:02:44<00:00, 36.23it/s]
/home/s192851/.conda/envs/sandbox/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [7]:
btc.to_pickle('sent_fbt.pickle')

In [60]:
df=pd.read_pickle('sent_fbt.pickle')
subs=pd.read_pickle('../Datasets/submissions.pickle')
df['sent_fbt']=df.sent_fbt.progress_apply(lambda x: x[0][1]['score']- x[0][2]['score'] if x!='no_result' else 0)

100%|██████████| 527651/527651 [00:00<00:00, 669387.22it/s]


In [70]:
merge=pd.concat([df,subs]).drop_duplicates(subset='id').sort_index()

In [72]:
merge.to_pickle('../Datasets/submissions.pickle')